In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tqdm import tqdm

In [2]:
def scrape_poem_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    poem_links = []
    link_elements = soup.find_all('a', class_='row')
    
    for link in link_elements:
        href = link.get('href')
        poem_links.append(href)
    
    return poem_links

In [3]:
def scrape_poem(url, title):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    poem_div = soup.find('div', class_='gedicht-originaltext clearfix')
    poem_lines = [line.strip() for line in poem_div.stripped_strings]
    
    author_h1 = soup.find('h1', id='gedicht-autor')
    author_name = author_h1.find('a').text.strip() if author_h1 else ''
    
    poem_data = {
        'title': title,
        'gender': '',
        'author': author_name,
        'poem': {}
    }
    
    for idx, line in enumerate(poem_lines, start=1):
        poem_data['poem'][f'line.{idx}'] = {'text': line.strip()}
    
    return poem_data


def save_to_json(poem_data, file_name):
    with open(file_name, 'w', encoding='utf-8') as file:
        json.dump(poem_data, file, indent=4, ensure_ascii=False)


In [4]:
# Scrape the poem links
url = 'https://www.lyrikline.org/de/gedichte?query=&onlynewoff=&lang[]=de&translatorname=999999&category[]=66'
poem_links = scrape_poem_links(url)

# Create a "corpus" folder if it doesn't exist
corpus_folder = 'corpus'
os.makedirs(corpus_folder, exist_ok=True)

# Scrape and save the poems
for link in tqdm(poem_links, desc='Scraping poems', unit='poem'):
    title = link.split('/')[-1]
    poem_url = f'https://www.lyrikline.org{link}'
    poem_data = scrape_poem(poem_url, title)
    file_name = os.path.join(corpus_folder, f'{title}.json')
    save_to_json(poem_data, file_name)

Scraping poems: 100%|████████████████████████████████████████████████████████████████| 20/20 [00:27<00:00,  1.37s/poem]


In [5]:
with open("corpus/alemannische-mangos-15731.json", 'r', encoding="utf-8") as file:
        poem_data = json.load(file)
poem_data['poem']

{'line.1': {'text': 'Wenn die Zweige zum Boden geneigt sind kannst du'},
 'line.2': {'text': 'auf den Baum klettern immer weiter so wie man'},
 'line.3': {'text': 'die Leiter wegwirft auf der man steht wenn du hinsieht'},
 'line.4': {'text': 'sind die klein geschrumpften Brombeeren aus diesem Jahr'},
 'line.5': {'text': 'alemannische Mangos die immer noch süß schmecken'},
 'line.6': {'text': 'wenn du still bist hörst du wie das Blut in den Bahnen wandert'},
 'line.7': {'text': 'wie es Geschichten erzählt aus Jahren als du mit'},
 'line.8': {'text': 'Bussarden geflogen bist wenn wir still sind hören wir'},
 'line.9': {'text': 'unter der gefrorenen Erde Verbindungen wachsen von Buche'},
 'line.10': {'text': 'zu Buche sehen Wunden nicht heilen aber vernarben wenn wir'},
 'line.11': {'text': 'weitergehen sind die toten Hummeln wieder lebendig gibt es'},
 'line.12': {'text': 'noch Lichtungen auf denen Wegerich wächst um ihn auf Schnitte zu legen'},
 'line.13': {'text': 'wenn wir die Gesicht